# **Sound Spot**

##### Installation et importations

In [2]:
# %pip install spotipy
# %pip install selenium
# pip install beautifulsoup4
# %pip install pandas
# %pip install --upgrade pip
# %pip install bcrypt
# %pip install mysql-connector-python
# %pip install librosa
# TODO: LIBROSA!!!
import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

import pandas as pd
import numpy as np
# import json

import requests

from selenium import webdriver

import sys
sys.path.append('GitHub\\data-camp\\secured_passwd.py')
from secured_passwd import CLIENT_ID, CLIENT_SECRET, SQL_PASSWD, GENIUS_ID, GENIUS_SECRET_ID

import re
import time

import lyricsgenius

import mysql.connector

Authorization Site : https://accounts.spotify.com/fr/authorize?client_id=ee862f2c84fe4f00929c71fb2eb50807&response_type=code&redirect_uri=https%3A%2F%2Fopen.spotify.com&scope=playlist-read-private

In [3]:
# IDs
client_id = CLIENT_ID
client_secret = CLIENT_SECRET

genius_id = GENIUS_ID
genius_secret_id = GENIUS_SECRET_ID

##### Accéder aux infos Spotify de l'utilisateurs pour créer notre BDD

In [4]:
# Créez une instance de Spotipy avec des identifiants client
sp_oauth = SpotifyOAuth(
    client_id = client_id,
    client_secret = client_secret,
    redirect_uri = 'https://open.spotify.com',
    scope = 'user-library-read',  # Autorisations nécessaires, y compris la lecture des playlists privées
    cache_path='.cache'
)


In [5]:
def mysql_connexion():
    # Create a connexion to the database
    connexion = mysql.connector.connect(
        host='192.168.1.76',
        user='soundspothandler',
        password=SQL_PASSWD,
        database='sound_spot'
    )

    # Create a cursor to execute SQL queries
    cursor = connexion.cursor()

    return cursor, connexion

In [6]:
# util.prompt_for_user_token(username,scope,client_id='your-spotify-client-id',client_secret='your-spotify-client-secret',redirect_uri='your-app-redirect-url')
def spotify_access_token():
    # Obtenir le jeton d'accès ou le renouveler s'il est expiré
    token_info = sp_oauth.get_cached_token()
    print("TOKEN EXISTS : ", token_info)

    # Vérifiez si le jeton d'accès est valide
    if not token_info:

        auth_url = sp_oauth.get_authorize_url()

        print(auth_url)

        driver = webdriver.Chrome()

        driver.get(auth_url)

        # Attendez que l'utilisateur autorise l'application et soit redirigé
        input("Appuyez sur Entrée une fois redirigé vers l'URL de redirection...")

        final_url = driver.current_url

        driver.quit()
        
        response = requests.get(final_url)
        print('RESPONSE : ', response.url)
        code = sp_oauth.get_authorization_code(response.url)
        print("CODE : ", code)

        access_token = sp_oauth.get_access_token(code, as_dict = False)
        # print("TOKEN : ", access_token)
    else:
        access_token = token_info['access_token']
    return access_token

In [7]:
def add_to_sql_db():

    cursor, connexion = mysql_connexion()

    # Convertir la liste en une chaîne de caractères
    other_artists_str = ', '.join(other_artists)

    # Convertir la liste des genres en une chaîne de caractères
    genres_str = ', '.join(main_artist_info['genres'])

    query_insert_into_main_artist = "INSERT INTO MAIN_ARTIST (main_artist_id, main_artist_name, main_artist_popularity, main_artist_genre, main_artist_image_url) VALUES (%s, %s, %s, %s, %s)"
    query_insert_into_track = "INSERT INTO TRACK (track_id, main_artist_id, released_date, loudness, tempo, tempo_confidence, time_signature, time_signature_confidence, track_name, track_popularity, other_artists, track_image_url, lyrics) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    
    values_in_main_artist = (main_artist_id, main_artist_info['name'], main_artist_info['popularity'], genres_str, main_artist_image_url)
    values_in_track = (track_info['id'], main_artist_id, track_info['album']['release_date'], loudness, tempo, tempo_confidence, time_signature, time_signature_confidence, track_name, track_info['popularity'], other_artists_str, track_info['album']['images'][0]['url'], get_lyrics)

    try:
        cursor.execute(query_insert_into_main_artist, values_in_main_artist)
        # cursor.execute(query_insert_into_track, values_in_track)
        connexion.commit()
    except mysql.connector.Error as err:
        print(f"Erreur lors de l'insertion : {err}")

    try:
        # cursor.execute(query_insert_into_main_artist, values_in_main_artist)
        cursor.execute(query_insert_into_track, values_in_track)
        connexion.commit()
    except mysql.connector.Error as err:
        print(f"Erreur lors de l'insertion : {err}")
    
    # Vérifier si les paroles existent
    existing_lyrics_query = f"SELECT lyrics FROM TRACK WHERE lyrics = 'Aucunes paroles trouvées pour cette chanson.'"
    cursor.execute(existing_lyrics_query, (track_info['id'],))
    existing_lyrics = cursor.fetchone()

    if existing_lyrics[0] is None:
        # Si les paroles n'existent pas, les insérer
        lyrics = fetch_lyrics(artist_name, track_name, new_access_token)
        update_query = "UPDATE TRACK SET lyrics = %s WHERE track_id = %s"
        cursor.execute(update_query, (lyrics, track_info['id']))
    else:
        # Les paroles existent déjà, vous pouvez les laisser en l'état ou effectuer d'autres actions selon vos besoins.
        pass

    # Commit pour enregistrer les modifications
    connexion.commit()

    # Fermer le curseur et la connexion
    cursor.close()
    connexion.close()

In [8]:
def genius_token():

    # Les informations d'authentification de votre application
    client_id = GENIUS_ID
    client_secret = GENIUS_SECRET_ID

    # Params to obtain a new access token
    token_url = 'https://api.genius.com/oauth/token'
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }

    # Do a POST request to have a new access token
    response = requests.post(token_url, data=data)

    if response.status_code == 200:
        # Access token is in the response
        new_access_token = response.json()['access_token']
        # print(f"Nouveau jeton d'accès : {new_access_token}")
        return new_access_token
    else:
        print("Échec de la génération du jeton d'accès.")

In [9]:
# S'authentifier avec le jeton d'accès
sp = spotipy.Spotify(auth=spotify_access_token())

max_retries = 3  # Nombre maximal de tentatives
retry_delay = 5  # Délai en secondes entre les tentatives

# ID de la playlist Spotify que vous souhaitez récupérer (vous pouvez obtenir l'ID depuis l'URL de la playlist)
playlist_id = 'https://open.spotify.com/playlist/4S1iqtui4d2kDtezNGWIgL?si=aa701d0a71fa4355&pt=123ef798a2e646a0619d8240d2f35821' # Private Playlist
# playlist_id = 'https://open.spotify.com/playlist/37i9dQZEVXbhGLvgw0jIZm?si=d1e5dfd1966c45b8' # Public Playlist

# Récupérez les informations sur la playlist
playlist = sp.playlist(playlist_id)

# Imprimez les informations sur la playlist
print(f"Nom de la playlist : {playlist['name']}")
print(f"Propriétaire de la playlist : {playlist['owner']['display_name']}")
print(f"Nombre de pistes dans la playlist : {playlist['tracks']['total']}")

# Liste des noms des pistes (en paginant)
track_data = []
results = sp.playlist_tracks(playlist_id)

column_names = [
    'Track ID', 'Main Artist ID', 'Track Name', 'Track Popularity',
    'Track Released Date', 'Loudness', 'Tempo', 'Tempo Confidence',
    'Time Signature', 'Time Signature Confidence', 'Main Artist',
    'Main Artist Genre', 'Main Artist Popularity', 'Other Artists',
    'Track URL', 'Track Image URL', 'Main Artist Image URL', 'Lyrics'
]
# data = pd.DataFrame()

while results:
    sp = spotipy.Spotify(auth=spotify_access_token())
    for _ in range(max_retries):
        try:
            for track in results['items']:

                # Track info
                track_info = track['track']

                # Audio
                audio = sp.audio_analysis(track_info['id'])
                audio = audio['track']

                loudness = audio['loudness']
                tempo = audio['tempo']
                tempo_confidence = audio['tempo_confidence']
                time_signature = audio['time_signature']
                time_signature_confidence = audio['time_signature_confidence']

                # Artists info
                artists = track['track']['artists']
                other_artists = []

                main_artist_id = track_info['artists'][0]['id']
                main_artist_info = sp.artist(main_artist_id)

                for artist in artists:
                    other_artists.append(artist['name'])
                
                other_artists.pop(0)

                if 'images' in main_artist_info and main_artist_info['images']:
                    main_artist_image_url = main_artist_info['images'][0]['url']
                elif 'image' in main_artist_info:
                    main_artist_image_url = main_artist_info['image']['url']
                else:
                    main_artist_image_url = None
                
                track_name = track_info['name'].replace(".", "")
                track_name = re.sub(r'\([^)]*\)', '', track_name)
                track_name = ' '.join(track_name.split())
                artist_name = main_artist_info['name'].replace(".", "")
          
                get_lyrics = fetch_lyrics(artist_name, track_name, genius_token())

                track_data.append([
                    track_info['id'],
                    main_artist_id,
                    track_info['name'],
                    track_info['popularity'],
                    track_info['album']['release_date'],
                    loudness,
                    tempo,
                    tempo_confidence,
                    time_signature,
                    time_signature_confidence,
                    main_artist_info['name'],
                    main_artist_info['genres'],
                    main_artist_info['popularity'],
                    other_artists,
                    track_info['external_urls'],
                    track_info['album']['images'][0]['url'],
                    main_artist_image_url,
                    get_lyrics
                ])

                add_to_sql_db()
                # print(track_info['id'], main_artist_id)

            results = sp.next(results)  # Charge the next page
            break
        except Exception as e:
            print(f"Erreur lors de la récupération des données : {str(e)}")
            if _ < max_retries - 1:
                print(f"Réessai dans {retry_delay} secondes...")
                time.sleep(retry_delay)
            else:
                print("Nombre maximal de tentatives atteint. Abandon.")

for idx in track_data:

    data = pd.DataFrame(track_data, columns=column_names)


TOKEN EXISTS :  {'access_token': 'BQCsN76mAx8SNzc93ahJrL7PwImSa8ngsOSSGxgqjN_xwWeeOtyF_blqu1Lj8Z9wsSCevBK8QVQqfGpAK_OHOA9dkyHswB3rT9E_RkgEQirxg32gR6fAjpcKjY2PzhjEY2yXofQvOkbHPxNy3bzt5KVVquNhQFG_sjl9lKW5isOl15ejaSfGR8p2FboS4lGX5IT--fubYS8we8M', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': 'user-library-read', 'expires_at': 1698745201, 'refresh_token': 'AQA1KuNc15AwIB3zyFJmz4epi9YPKp-Fz_b3neDBulVt_K2mYEy5QlD3OG_9CHmHN9nhd-oKi6jWKG7HeCRO1NT2FoFJUvDN4wUYcKsyqSHjoecuKIDMuFHXA75NQhPanuE'}
Nom de la playlist : My recommendation playlist
Propriétaire de la playlist : Chelsie
Nombre de pistes dans la playlist : 10
TOKEN EXISTS :  {'access_token': 'BQCsN76mAx8SNzc93ahJrL7PwImSa8ngsOSSGxgqjN_xwWeeOtyF_blqu1Lj8Z9wsSCevBK8QVQqfGpAK_OHOA9dkyHswB3rT9E_RkgEQirxg32gR6fAjpcKjY2PzhjEY2yXofQvOkbHPxNy3bzt5KVVquNhQFG_sjl9lKW5isOl15ejaSfGR8p2FboS4lGX5IT--fubYS8we8M', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': 'user-library-read', 'expires_at': 1698745201, 'refresh_token': 'AQA1KuNc15AwIB3zy

Max Retries reached


Erreur lors de la récupération des données : http status: 429, code:-1 - /v1/audio-analysis/0WH65R3DLtFPh8KN2b1mzI:
 Max Retries, reason: too many 429 error responses
Réessai dans 5 secondes...


Max Retries reached


Erreur lors de la récupération des données : http status: 429, code:-1 - /v1/audio-analysis/0WH65R3DLtFPh8KN2b1mzI:
 Max Retries, reason: too many 429 error responses
Réessai dans 5 secondes...


Max Retries reached


Erreur lors de la récupération des données : http status: 429, code:-1 - /v1/audio-analysis/0WH65R3DLtFPh8KN2b1mzI:
 Max Retries, reason: too many 429 error responses
Nombre maximal de tentatives atteint. Abandon.
TOKEN EXISTS :  {'access_token': 'BQCsN76mAx8SNzc93ahJrL7PwImSa8ngsOSSGxgqjN_xwWeeOtyF_blqu1Lj8Z9wsSCevBK8QVQqfGpAK_OHOA9dkyHswB3rT9E_RkgEQirxg32gR6fAjpcKjY2PzhjEY2yXofQvOkbHPxNy3bzt5KVVquNhQFG_sjl9lKW5isOl15ejaSfGR8p2FboS4lGX5IT--fubYS8we8M', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': 'user-library-read', 'expires_at': 1698745201, 'refresh_token': 'AQA1KuNc15AwIB3zyFJmz4epi9YPKp-Fz_b3neDBulVt_K2mYEy5QlD3OG_9CHmHN9nhd-oKi6jWKG7HeCRO1NT2FoFJUvDN4wUYcKsyqSHjoecuKIDMuFHXA75NQhPanuE'}


Max Retries reached


Erreur lors de la récupération des données : http status: 429, code:-1 - /v1/audio-analysis/0WH65R3DLtFPh8KN2b1mzI:
 Max Retries, reason: too many 429 error responses
Réessai dans 5 secondes...


KeyboardInterrupt: 

##### Transform tables into dataframe and apply Machine Learning Models

In [43]:
def to_dataframe():
    cursor, connexion = mysql_connexion()

    # SQL query to select data from a table
    track_table_df_query = "SELECT * FROM track"
    artist_table_df_query = "SELECT * FROM main_artist"

   # Load the table into a Pandas DataFrame
    track_table = pd.read_sql(track_table_df_query, connexion)
    artist_table = pd.read_sql(artist_table_df_query, connexion)

    # Close the database connection
    connexion.close()

    return track_table, artist_table

track_table, artist_table = to_dataframe()

C:\Users\louis\AppData\Local\Temp\ipykernel_22352\391624903.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  track_table = pd.read_sql(track_table_df_query, connexion)
C:\Users\louis\AppData\Local\Temp\ipykernel_22352\391624903.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  artist_table = pd.read_sql(artist_table_df_query, connexion)


In [49]:
data = pd.merge(track_table, artist_table, on='main_artist_id')
data.to_csv('sound_spot_database.csv', index=False)
track_table.to_csv("track_table.csv", index=False)

In [53]:
from sklearn.preprocessing import LabelEncoder

# Créez une instance de LabelEncoder
label_encoder = LabelEncoder()

# Liste des colonnes à encoder
columns_to_encode = ['track_name', 'other_artists', 'lyrics', 'main_artist_name', 'main_artist_genre']

# Parcourez les colonnes et encodez-les
for col in columns_to_encode:
    data[col] = label_encoder.fit_transform(data[col])

from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity between tracks based on selected features
features = ['loudness', 'track_popularity', 'track_name',
            'track_popularity', 'other_artists', 'lyrics',
            'main_artist_name', 'main_artist_popularity',
            'main_artist_genre']

data_features = data[features].fillna(0)
cosine_sim = cosine_similarity(data_features)

# Get recommendations for a specific track
track_name = "Mourning"
track_idx = track_table[track_table['track_name'] == track_name].index[0]

# Get the most similar tracks
similar_tracks = list(enumerate(cosine_sim[track_idx]))
similar_tracks = sorted(similar_tracks, key=lambda x: x[1], reverse=True)

# Print the top recommended tracks
num_recommendations = 5
for i in range(1, num_recommendations + 1):
    similar_track = track_table.iloc[similar_tracks[i][0]]['track_name']
    similarity_score = similar_tracks[i][1]  # Score de similarité
    print(f"Recommendation {i}: {similar_track} (Similarity Score: {similarity_score})")


Recommendation 1: You Right (Similarity Score: 0.9988631454616692)
Recommendation 2: NASA (Similarity Score: 0.980074553989174)
Recommendation 3: The Great British Bar Off (Similarity Score: 0.9712339266763266)
Recommendation 4: Codé (Similarity Score: 0.9683889072551598)
Recommendation 5: I Should've Known (Similarity Score: 0.9614071294917977)


In [41]:
data

,track_id,main_artist_id,released_date,loudness,tempo,tempo_confidence,time_signature,time_signature_confidence,track_name,track_popularity,other_artists,track_image_url,lyrics,main_artist_name,main_artist_popularity,main_artist_genre,main_artist_image_url
0,02WbmXcfVEPDOZH91K8OtL,6jGMq4yGs7aQzuGsMgVgZR,2023-10-20,-8.524,130.041,0.110,4.0,1.000,69,66,34,https://i.scdn.co/image/ab67616d0000b273e8de6f...,36,66,78,21,https://i.scdn.co/image/ab6761610000e5eb61a229...
1,0m9cSkuwYOCUWCf48IS5ds,6jGMq4yGs7aQzuGsMgVgZR,2023-07-07,-6.539,137.487,0.163,4.0,1.000,119,65,66,https://i.scdn.co/image/ab67616d0000b273577fe7...,87,66,78,21,https://i.scdn.co/image/ab6761610000e5eb61a229...
2,045o0OHz10mgb98wPkq1eO,75VKfyoBlkmrJFDqo1o2VY,2023-10-20,-6.472,125.961,0.726,4.0,1.000,176,38,9,https://i.scdn.co/image/ab67616d0000b2738e03af...,4,34,69,1,https://i.scdn.co/image/ab6761610000e5eb883c07...
3,05mDaV9Vb3wrzjF6OPZnhq,246dkjvS1zLTtiykXe5h60,2019-09-06,-3.618,132.113,0.534,4.0,0.422,137,74,0,https://i.scdn.co/image/ab67616d0000b2739478c8...,93,83,87,41,https://i.scdn.co/image/ab6761610000e5eb6be070...
4,0gWrMbx6pbdH3n3nsLjE55,246dkjvS1zLTtiykXe5h60,2023-05-19,-4.568,74.013,0.000,4.0,0.733,97,76,0,https://i.scdn.co/image/ab67616d0000b2736e9e05...,17,83,87,41,https://i.scdn.co/image/ab6761610000e5eb6be070...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,7EiZI6JVHllARrX9PUvAdX,1RyvyyTE3xzB2ZywiAwp0i,2016-04-13,-7.789,143.961,0.351,4.0,1.000,88,84,57,https://i.scdn.co/image/ab67616d0000b273626745...,80,37,87,10,https://i.scdn.co/image/ab6761610000e5eb24e41f...
184,7ezboaARv4FY1k0WFoINWC,0ErzCpIMyLcjPiwT4elrtZ,2023-02-03,-6.217,132.919,0.356,4.0,0.912,18,62,0,https://i.scdn.co/image/ab67616d0000b273868e59...,67,74,77,68,https://i.scdn.co/image/ab6761610000e5ebcc5865...
185,7vNwsXLLZIZfJifJxzuqXU,0ErzCpIMyLcjPiwT4elrtZ,2023-10-23,-4.844,130.934,0.785,4.0,0.972,20,65,0,https://i.scdn.co/image/ab67616d0000b2738e7254...,179,74,77,68,https://i.scdn.co/image/ab6761610000e5ebcc5865...
186,7hxHWCCAIIxFLCzvDgnQHX,6MPCFvOQv5cIGfw3jODMF0,2020-08-28,-6.142,140.042,0.609,4.0,1.000,84,75,21,https://i.scdn.co/image/ab67616d0000b2739ca0c9...,133,43,66,71,https://i.scdn.co/image/ab6761610000e5eb30cc15...


In [59]:
# to_decode = 'track_name'
# track_table[to_decode] = label_encoder.inverse_transform(track_table[to_decode])

,track_id,main_artist_id,released_date,loudness,tempo,tempo_confidence,time_signature,time_signature_confidence,track_name,track_popularity,other_artists,track_image_url,lyrics
0,02WbmXcfVEPDOZH91K8OtL,6jGMq4yGs7aQzuGsMgVgZR,2023-10-20,-8.524,130.041,0.110,4.0,1.000,I Should've Known,66,Kyle Richh,https://i.scdn.co/image/ab67616d0000b273e8de6f...,"[Intro: Lil Tjay]<br>Mm-mmm-mm<br>Mm<br>Ayy, N..."
1,045o0OHz10mgb98wPkq1eO,75VKfyoBlkmrJFDqo1o2VY,2023-10-20,-6.472,125.961,0.726,4.0,1.000,YAWA - Lagos Mix,38,Clayton William,https://i.scdn.co/image/ab67616d0000b2738e03af...,Aucunes paroles trouvées pour cette chanson.
2,05mDaV9Vb3wrzjF6OPZnhq,246dkjvS1zLTtiykXe5h60,2019-09-06,-3.618,132.113,0.534,4.0,0.422,Saint-Tropez,74,,https://i.scdn.co/image/ab67616d0000b2739478c8...,"[Intro]<br>Ooh, yeah, yeah, hey<br><br>[Chorus..."
3,0cqRj7pUJDkTCEsJkx8snD,06HL4z0CvFAxyc27GXpf02,2014-01-01,-5.384,160.078,0.257,4.0,0.974,Shake It Off,87,,https://i.scdn.co/image/ab67616d0000b27352b2a3...,[Verse 1]<br>I stay out too late<br>Got nothin...
4,0D4KNiAvIR3lQrYihugqJc,1URnnhqYAYcrqrcwql10ft,2023-10-17,-6.298,158.942,0.121,4.0,0.947,Call Me Revenge - Call of Duty: Modern Warfare 3,56,d4vd,https://i.scdn.co/image/ab67616d0000b273b41a09...,[Chorus: d4vd]<br>Call me revenge<br>Cold as t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,7o42ikuu4a51sk1dp1Kxsv,4YUcg2ee9TxMNdjhAxzZBF,2022-01-13,-10.010,113.081,0.428,4.0,0.982,SEIZE,25,,https://i.scdn.co/image/ab67616d0000b2738bc9bd...,"[Paroles de ""SEIZE""]<br><br>[Intro]<br>Oh, oh,..."
184,7pdF27mSDuPWhppnHAmWHa,15UsOTVnJzReFVN1VCnxy4,2018-12-07,-2.939,160.147,0.840,4.0,1.000,whoa,73,,https://i.scdn.co/image/ab67616d0000b273da9e59...,[Intro]<br>I was like whoa-oh-oh<br>Whoa-oh-oh...
185,7tzOeM0LbXJYMzjny2D45r,6dlzQ6fiPna40trq1Ek6cb,2022-06-03,-6.033,102.754,0.765,4.0,1.000,Summer Love,54,,https://i.scdn.co/image/ab67616d0000b273165c1e...,"Believe, Believe, Believe<br>(chant)<br>MICHON..."
186,7vNwsXLLZIZfJifJxzuqXU,0ErzCpIMyLcjPiwT4elrtZ,2023-10-23,-4.844,130.934,0.785,4.0,0.972,CMON FREESTYLE,65,,https://i.scdn.co/image/ab67616d0000b2738e7254...,"[Verse]<br>Ayy, them boy better get out this s..."


In [54]:
data[data['track_name'] == 'A Keeper']

,track_id,main_artist_id,released_date,loudness,tempo,tempo_confidence,time_signature,time_signature_confidence,track_name,track_popularity,other_artists,track_image_url,lyrics,main_artist_name,main_artist_popularity,main_artist_genre,main_artist_image_url


In [63]:
data = pd.read_csv('sound_spot_database.csv', delimiter=',')
track_table = pd.read_csv('track_table.csv', delimiter=',')

chanson = "A Keeper"

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

columns_to_encode = ['track_name', 'other_artists', 'lyrics', 'main_artist_name', 'main_artist_genre']

for col in columns_to_encode:
    data[col] = label_encoder.fit_transform(data[col])

# Calculate cosine similarity between tracks based on selected features
features = ['loudness', 'track_popularity', 'track_name',
            'track_popularity', 'other_artists', 'lyrics',
            'main_artist_name', 'main_artist_popularity',
            'main_artist_genre']

data_features = data[features].fillna(0)
cosine_sim = cosine_similarity(data_features)

# Find the index of the given song
track_idx = track_table[track_table['track_name'] == chanson].index

if not track_idx.empty:
    track_idx = track_idx[0]  # Get the first index if there are multiple matches

    # Get the most similar tracks
    similar_tracks = list(enumerate(cosine_sim[track_idx]))
    similar_tracks = sorted(similar_tracks, key=lambda x: x[1], reverse=True)

    # Extract the top recommended tracks
    recommended_tracks = []
    for i in range(1, num_recommendations + 1):
        similar_track = track_table.iloc[similar_tracks[i][0]]['track_name']
        similarity_score = similar_tracks[i][1]
        recommended_tracks.append({
            "track_name": similar_track,
            "similarity_score": similarity_score
        })

    print(recommended_tracks)

else:
    print('error: Track not found')

[{'track_name': 'THRU THE NIGHT', 'similarity_score': 0.9631275970827734}, {'track_name': 'Chosen', 'similarity_score': 0.9530087888333754}, {'track_name': 'Sale histoire', 'similarity_score': 0.9328553807196751}, {'track_name': 'First Person Shooter', 'similarity_score': 0.9200404265322136}, {'track_name': 'Undecided', 'similarity_score': 0.9143182659752163}]


##### **Retrouver les infos d'une chanson juste en tapant le nom de la chanson et celui de l'artiste**

In [ ]:
# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials

# # Configurez l'authentification
# sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# # Remplacez ces valeurs par le nom de la chanson et de l'artiste que vous recherchez
# chanson = 'The Hills'
# artiste = 'The Weeknd'

# # Recherchez la chanson
# results = sp.search(q=f'track:{chanson} artist:{artiste}', type='track')

# # Obtenez les données de la première chanson de la liste de résultats
# if len(results['tracks']['items']) > 0:
#     chanson_data = results['tracks']['items'][0]
#     print('Nom de la chanson:', chanson_data['name'])
#     print('Artiste:', chanson_data['artists'][0]['name'])
#     print('Album:', chanson_data['album']['name'])
#     print('URL Spotify:', chanson_data['external_urls']['spotify'])
# else:
#     print('Chanson non trouvée.')


In [ ]:
# Nom de l'album, de l'artiste
# Type de chanson, rythmique, popularité
# Lyrics --> Webscrapping

# À voir : category, category_playlists, featured_playlists
# playlist_add_items, playlist_change_details, artist_albums
# artist_related_artists, artist_top_tracks, audio_analysis,
# audio_features, playlist_cover_image, playlist_items
# recommendation_genre_seeds, recommandations, search,
# seek_track, user_playlists, get_access_token...

# Possibilité d'écouter de l'audio depuis le site

# Filtre pour ranger dans un certain ordre



# ................................................................
# # Retrieve the user's top tracks
# top_tracks = sp.current_user_top_tracks(limit=10, time_range='medium_term')  # You can adjust the limit and time_range

# # Print the top tracks
# for idx, track in enumerate(top_tracks['items'], start=1):
#     print(f"{idx}. {track['name']} by {', '.join([artist['name'] for artist in track['artists']])}")